In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import time
import matplotlib
matplotlib.rcParams['toolbar'] = 'None' 
import matplotlib.pyplot as plt

# model path
model_path = 'model.h5'

# data
cifar10 = tf.keras.datasets.cifar10
((x_train, y_train), (x_test, y_test)) = cifar10.load_data()
# Eliminate an unused dimension
y_train, y_test = np.reshape(y_train, y_train.shape), np.reshape(y_test, y_test.shape)

# label names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# training hyperparameters
batch_size = 32
learning_rate = 0.001
optimizer = 'Adam'
epochs = 20

# structure of the model
model = tf.keras.models.Sequential([
    tf.keras.applications.VGG16(include_top=False, weights=None, input_shape=(32, 32, 3)),
    tf.keras.layers.Flatten(), # flatten
    tf.keras.layers.Dense(4096, activation='relu'), # FC1
    tf.keras.layers.Dense(4096, activation='relu'), # FC2
    tf.keras.layers.Dense(10, activation='softmax'), # output layer (with softmax)
])

In [ ]:
model.summary()

In [ ]:
# choose a optimizer
if optimizer.upper() == 'ADAM':
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
elif optimizer.upper() == 'SGD':
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

# compile the model, prepare to train
model.compile(
    optimizer=opt,
    #optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), # from_logits=False: output layer is already softmax
    metrics=['accuracy']
)

# normalized  the data
x_train_n, x_test_n = x_train / 255.0, x_test / 255.0
y_train, y_test = y_train, y_test

# training
history_training = model.fit(x_train_n, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test_n, y_test))

# save model
model.save(model_path)

# plot
training_accuracy = np.array(history_training.history['accuracy']) * 100
training_loss = history_training.history['loss']
testing_accuracy = np.array(history_training.history['val_accuracy']) * 100

plt.figure('Accuracy and Loss')
plt.subplot(2, 1, 1)
plt.title('Accuracy')
plt.plot(training_accuracy, label='Training')
plt.plot(testing_accuracy, label = 'Testing')
plt.ylabel('%')
plt.legend(loc='lower right')
plt.subplot(2, 1, 2)
plt.plot(training_loss)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()
t = time.localtime()
plt.savefig('history_' + time.strftime("%Y%m%d_%H%M%S", t) + '.png')